In [ ]:
"# Databricks notebook source\n# MAGIC %md\n# MAGIC # ETL Process for Customer 360 Data\n# MAGIC This notebook performs an ETL process to integrate and transform data from various sources into a comprehensive Customer 360 view.\n\n# COMMAND ----------\n\nimport logging\nfrom pyspark.sql.functions import col, count, max, avg, datediff, current_date, when, lit, broadcast\n\n# Configure logging\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\n# Assume the Spark session is pre-initialized as 'spark'\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 1: Data Source Configuration\n# MAGIC Load data from Unity Catalog tables.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Loading data sources from Unity Catalog tables.\")\n    policy_df = spark.table(\"catalog.db.policy\")\n    claims_df = spark.table(\"catalog.db.claims\")\n    demographics_df = spark.table(\"catalog.db.demographics\")\n    scores_df = spark.table(\"catalog.db.scores\")\n    aiml_insights_df = spark.table(\"catalog.db.aiml_insights\")\nexcept Exception as e:\n    logger.error(f\"Error loading data sources: {e}\")\n    raise\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 2: Data Selection and Filtering\n# MAGIC Select relevant fields from the data sources.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Selecting relevant fields from data sources.\")\n    demographics_selected = demographics_df.select(\n        \"Customer_ID\", \"Customer_Name\", \"Email\", \"Phone_Number\", \"Address\", \"City\", \"State\", \"Postal_Code\",\n        \"Date_of_Birth\", \"Gender\", \"Marital_Status\", \"Occupation\", \"Income_Level\", \"Customer_Segment\"\n    )\n    claims_selected = claims_df.select(\n        \"Claim_ID\", \"Policy_ID\", \"Claim_Date\", \"Claim_Type\", \"Claim_Status\", \"Claim_Amount\", \"Claim_Payout\"\n    )\nexcept Exception as e:\n    logger.error(f\"Error in data selection and filtering: {e}\")\n    raise\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 3: Data Integration\n# MAGIC Join datasets using broadcast join for small dimension tables.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Joining datasets.\")\n    customer_policy_joined = demographics_selected.join(\n        broadcast(policy_df), demographics_selected.Customer_ID == policy_df.customer_id, \"inner\"\n    )\n    \n    full_joined_df = customer_policy_joined.join(\n        claims_selected, customer_policy_joined.policy_id == claims_selected.Policy_ID, \"inner\"\n    )\nexcept Exception as e:\n    logger.error(f\"Error in data integration: {e}\")\n    raise\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 4: Data Aggregation\n# MAGIC Aggregate data to compute metrics like total claims, policy count, etc.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Aggregating data.\")\n    aggregated_df = full_joined_df.groupBy(\"Customer_ID\").agg(\n        count(\"Claim_ID\").alias(\"Total_Claims\"),\n        count(\"policy_id\").alias(\"Policy_Count\"),\n        max(\"Claim_Date\").alias(\"Recent_Claim_Date\"),\n        avg(\"Claim_Amount\").alias(\"Average_Claim_Amount\")\n    )\nexcept Exception as e:\n    logger.error(f\"Error in data aggregation: {e}\")\n    raise\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 5: Custom Calculations\n# MAGIC Perform custom calculations to derive additional insights.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Performing custom calculations.\")\n    final_df = aggregated_df.withColumn(\"Age\", datediff(current_date(), col(\"Date_of_Birth\")) / 365) \\\n        .withColumn(\"Claim_To_Premium_Ratio\", when(col(\"total_premium_paid\") != 0, col(\"Average_Claim_Amount\") / col(\"total_premium_paid\")).otherwise(0)) \\\n        .withColumn(\"Claims_Per_Policy\", when(col(\"Policy_Count\") != 0, col(\"Total_Claims\") / col(\"Policy_Count\")).otherwise(0)) \\\n        .withColumn(\"Retention_Rate\", lit(0.85)) \\\n        .withColumn(\"Cross_Sell_Opportunities\", lit(\"Multi-Policy Discount, Home Coverage Add-on\")) \\\n        .withColumn(\"Upsell_Potential\", lit(\"Premium Vehicle Coverage\"))\nexcept Exception as e:\n    logger.error(f\"Error in custom calculations: {e}\")\n    raise\n\n# COMMAND ----------\n\n# MAGIC %md\n# MAGIC ## Step 6: Output Configuration\n# MAGIC Write the final DataFrame to a Unity Catalog table.\n\n# COMMAND ----------\n\ntry:\n    logger.info(\"Writing output to Unity Catalog table.\")\n    final_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"catalog.target_db.customer_360\")\nexcept Exception as e:\n    logger.error(f\"Error writing output: {e}\")\n    raise\n\nlogger.info(\"ETL process completed successfully.\")\n"
